In [10]:
import pandas as pd
import numpy as np
import torch
import transformers
import bitsandbytes as bnb
import os
import wandb
import json

from transformers import AdamW, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from tqdm import tqdm
from peft import get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit, PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments,BitsAndBytesConfig
num=1

import streamlit as st
import pandas as pd
from loguru import logger

from langchain.chains import ConversationalRetrievalChain
#이 체인은 메모리를 가지고 있는 체인이다
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
# 기본 라이브러리 임포트

# PyTorch 관련

from torch import torch
from torch import cuda, bfloat16

# Transformers 관련
import transformers
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments,
    GenerationConfig,
    pipeline,
)
import pandas as pd
import numpy as np
import torch
import transformers
import bitsandbytes as bnb
import os
import wandb
import json

from transformers import AdamW, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from tqdm import tqdm
from peft import get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit, PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments,BitsAndBytesConfig
num=1

import streamlit as st
import pandas as pd
from loguru import logger

from langchain.chains import ConversationalRetrievalChain
#이 체인은 메모리를 가지고 있는 체인이다
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
# 기본 라이브러리 임포트

# PyTorch 관련

from torch import torch
from torch import cuda, bfloat16

# Transformers 관련
import transformers
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments,
    GenerationConfig,
    pipeline,
)
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments,BitsAndBytesConfig
from datasets import load_dataset
from trl import SFTTrainer
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training


# LangChain 관련
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import HuggingFacePipeline
from langchain_community.document_loaders import JSONLoader
# LangChain Community 관련
import json
from pathlib import Path
from pprint import pprint
from langchain_community.document_loaders import DataFrameLoader

from langchain.memory import ConversationBufferMemory
#메모리를 가진 체인을 구현하기 위해서는 ConversationBufferMemory를 통해 몇개까지 기억할 것인가를 정함
from langchain.vectorstores import FAISS
from torch import cuda, bfloat16
# from streamlit_chat import message
#아래는 둘다 메모리 기억을 위한 라이브러리
from langchain.callbacks import get_openai_callback
from langchain.memory import StreamlitChatMessageHistory
# LangChain 관련
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import HuggingFacePipeline
from langchain_community.document_loaders import JSONLoader
# LangChain Community 관련
import json
from pathlib import Path
from pprint import pprint
from langchain_community.document_loaders import DataFrameLoader

from langchain.memory import ConversationBufferMemory
#메모리를 가진 체인을 구현하기 위해서는 ConversationBufferMemory를 통해 몇개까지 기억할 것인가를 정함
from langchain.vectorstores import FAISS
from torch import cuda, bfloat16
# from streamlit_chat import message
#아래는 둘다 메모리 기억을 위한 라이브러리
from langchain.callbacks import get_openai_callback
from langchain.memory import StreamlitChatMessageHistory

# fine_tuning 모델 로드

In [11]:
#qlora모델 로드하는 코드
adapter_dir = f'./results{num}/final_checkpoint'
output_dir = f'./merged_peft{num}'
output_merged_dir = os.path.join(output_dir, "final_merged_checkpoint")

DEV = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)


model_name =output_merged_dir
adapter_path = f'./results{num}/final_checkpoint'
# adapter_path = "./dpo_results/final_checkpoint"

model = AutoPeftModelForCausalLM.from_pretrained(
    adapter_path,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    low_cpu_mem_usage=True
)

tokenizer = AutoTokenizer.from_pretrained(output_merged_dir)
tokenizer.eos_token='</s>'

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# vector_db 로드

In [12]:
# 벡터스토어에 임베딩할 모델 선언
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
model_name = 'intfloat/multilingual-e5-large'
model_kwargs = {"device":"cuda"}
encode_kwargs={'normalize_embeddings':True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs

)

In [13]:
#저장된 벡터 db에서 불러오기
vector_db = Chroma(persist_directory="./version_10", embedding_function=embeddings)

# Pipeline 로드

In [14]:
#허깅페이스 pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import LlamaForCausalLM, AutoTokenizer, pipeline
pipe = pipeline("text-generation", model=model, max_new_tokens=512,tokenizer=tokenizer, device_map="auto")
llm = HuggingFacePipeline(pipeline=pipe)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

# 랭체인 구성

In [15]:
#랭체인 구성
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=vector_db.as_retriever(search_type = 'mmr',search_kwargs={'k':3,'fetch_k':3}, vervose = True),
        memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key='answer'),
        get_chat_history=lambda h: h,
        return_source_documents=True,
        verbose = True
    )
conversation_chain.combine_docs_chain.llm_chain.prompt.template = """너는 논문 검색 도우미야. 맥락을 참고해서 답해줘. 답변의 시작은 "관련된 논문으로 ~~가 있습니다" 로 해줘.if you can't answer, please say 잘 모르겠습니다.

    {context}

    질문: {question}

    논문 검색 결과:"""


In [16]:
# 숫자 바꾸면 대화 바뀜
answer=conversation_chain('경제학에 대한 논문을 찾아줘')
answer['chat_history'][1].content

/home/t16user1/.conda/envs/ahdgks/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
너는 논문 검색 도우미야. 맥락을 참고해서 답해줘. 답변의 시작은 "관련된 논문으로 ~~가 있습니다" 로 해줘.if you can't answer, please say 잘 모르겠습니다.

    이 논문의 제목은 "경제학의 기본개념을 통한 경제교육 방안"이다. 저자는 "김진영"이다. 발행연도는 "2007"이다.논문의 내용은 경제 소양 혹은 경제 이해력은 개인과 국가 경쟁력을 결정하는 요인으로 경제교육의 필요성에 대해 이의를 제기하는 사람은 없다. 그러나 경제교육에서 어떻게, 무엇을 가르쳐야 하는 것인지는 밝혀져야 할 내용이 더 많다.  이다.

이 논문의 제목은 "경제학의 기본개념을 통한 경제교육 방안"이다. 저자는 "김진영"이다. 발행연도는 "2007"이다.논문의 내용은 경제 소양 혹은 경제 이해력은 개인과 국가 경쟁력을 결정하는 요인으로 경제교육의 필요성에 대해 이의를 제기하는 사람은 없다. 그러나 경제교육에서 어떻게, 무엇을 가르쳐야 하는 것인지는 밝혀져야 할 내용이 더 많다.  이다.

이 논문의 제목은 "에 나타난 유학사상의 경제관 연구"이다. 저자는 "김조영"이다. 발행연도는 "2016"이다.논문의 내용은 본 논문은 경제적 가치를 최고로 여기는 근대 자본주의 경제관의 단점을 극복하기 위해, 인간다움의 정체성을 추구하는 유학의 경제관을 를 통해 분석하여 자본주의 경제관과 비교한 뒤 그 시사점을 밝히고 추구하고자 한다.  이다.

    질문: 경제학에 대한 논문을 찾아줘

    논문 검색 결과:

> Finished chain.

> Finished chain.


'\n\n    1. 경제학의  기본개념 을 통한 경제교육  방안 \n    저자: 김진영\n    발행연도: 2007\n\n    2. 에 나타난 유학사상 의 경제 관 연구 \n    저자: 김조영\n    발행연도: 2016'

In [18]:
answer

{'question': '경제학에 대한 논문을 찾아줘',
 'chat_history': [HumanMessage(content='경제학에 대한 논문을 찾아줘'),
  AIMessage(content='\n\n    1. 경제학의  기본개념 을 통한 경제교육  방안 \n    저자: 김진영\n    발행연도: 2007\n\n    2. 에 나타난 유학사상 의 경제 관 연구 \n    저자: 김조영\n    발행연도: 2016')],
 'answer': '\n\n    1. 경제학의  기본개념 을 통한 경제교육  방안 \n    저자: 김진영\n    발행연도: 2007\n\n    2. 에 나타난 유학사상 의 경제 관 연구 \n    저자: 김조영\n    발행연도: 2016',
 'source_documents': [Document(page_content='이 논문의 제목은 "경제학의 기본개념을 통한 경제교육 방안"이다. 저자는 "김진영"이다. 발행연도는 "2007"이다.논문의 내용은 경제 소양 혹은 경제 이해력은 개인과 국가 경쟁력을 결정하는 요인으로 경제교육의 필요성에 대해 이의를 제기하는 사람은 없다. 그러나 경제교육에서 어떻게, 무엇을 가르쳐야 하는 것인지는 밝혀져야 할 내용이 더 많다.  이다.', metadata={'date': 2007, 'doc_id': 'A201008117477', 'ipc': '사회과학', 'title': '경제학의 기본개념을 통한 경제교육 방안'}),
  Document(page_content='이 논문의 제목은 "경제학의 기본개념을 통한 경제교육 방안"이다. 저자는 "김진영"이다. 발행연도는 "2007"이다.논문의 내용은 경제 소양 혹은 경제 이해력은 개인과 국가 경쟁력을 결정하는 요인으로 경제교육의 필요성에 대해 이의를 제기하는 사람은 없다. 그러나 경제교육에서 어떻게, 무엇을 가르쳐야 하는 것인지는 밝혀져야 할 내용이 더 많다.  이다.', metadata={'date': 2007, 'doc_id': 'A20100811